In [ ]:
import os

# Specify the directory containing your custom modules
custom_module_path = ""

# Get the current value of PYTHONPATH (if any)
existing_pythonpath = os.environ.get("PYTHONPATH", "")

# Append your custom module path to PYTHONPATH
os.environ["PYTHONPATH"] = f"{custom_module_path}:{existing_pythonpath}"

# Now Python will search for modules in the specified directory
print(existing_pythonpath)